#  An Introduction to ROS and Python

Addition resource:
* Rospy Tutorials: http://wiki.ros.org/rospy/Tutorials

## 1. Workspace creation

For developing new packages in ROS, you first need to create a workspace in which to place your package code. The following commands creates a new Catkin workspace.

**`mkdir ~/workspace_ros
cd ~/workspace_ros
mkdir src
cd src
catkin_init_workspace
cd ..
catkin_make
source devel/setup.bash
`**



The last command adds some configuration to your terminal session to allow you to run your package. If you want to make this changes appear in every terminal, you can add the following line at the end of **~/.bashrc** file

**`source ~/workspace_ros/devel/setup.bash´**

## 1.2. Package creation

New ROS packages are placed in the **src** folder of your development workspace. Use **catkin_create_pkg** command to create a new package

**`cd ~/workspace_pybot/src
catkin_create_pkg <package_name> [depend1] [depend2] [depend3]
`**

### Basic depedendencies

* rospy
* std_msgs
* turtlebot_msgs
* sensor_msgs

### Mydemo package example

**`catkin_create_pkg mydemo rospy std_msgs turtlebot_msgs sensor_msgs`**

Next, go to newly create package folder and create an **scripts** folder to hold your Python code.

**`cd mydemo
mkdir scripts
cd scripts`**

For example, you can create a file named **mydemo_hello.py** with the following code that creates a new ROS node.


In [ ]:
#!/usr/bin/python

# mydemo_hello.py

import rospy

# node entry point
if __name__ == '__main__':
    
    print('mydemo_hello: initializing ROS node')
    
    # init ros node and start spinning
    rospy.init_node('mydemo_hello')
    rospy.spin()
    
    print('mydemo_hello: shut down')

You need the mark this file as an executable file

**`chmod +x mydemo_hello.py`**

Additionallym, it is necessary to inform ROS that this script is the 'entry point' of the node. We do this by adding it to the **install** section in the **CMakeLists.txt** file in you **mydemo** package folder. Locate the following text in the file and uncomment the install command and add your **mydemo_hello.py** file

*mydemo/CMakeLists.txt*
<pre>
...

## Mark executable scripts (Python etc.) for installation
## in contrast to setup.py, you can choose the destination
install(PROGRAMS
  scripts/mydemo_hello.py
  DESTINATION ${CATKIN_PACKAGE_BIN_DESTINATION}
)

...
</pre>

## 1.3. Package build

To build the new package, run the the **catkin_make** command in the root directory of your workspace

**`cd ~/workspace_ros
catkin_make`**

## 2. Basic ROS Node programming

The basic structure of a ROS node in Python looks as follows

In [ ]:
#!/usr/bin/python

# mydemo_hello.py

import rospy

# node entry point
if __name__ == '__main__':
    
    print('mydemo_hello: initializing ROS node')
    
    # init ros node and start spinning
    rospy.init_node('mydemo_hello')
    rospy.spin()
    
    print('mydemo_hello: shut down')

**rospy** module encapsulates all the ROS functions accessible from Python

## 3 Capture images under ROS

** `
cd ~/workspace_pybot/src
catkin_create_pkg pybot rospy geometry_msgs sensor_msgs cv_bridge
cd pybot
mkdir scripts
cd scripts
`**
### 3.1 Publisher

In [ ]:
#!/usr/bin/python
#
# The MIT License (MIT)
# 
# Copyright (c) 2015 Australian Centre for Robotic Vision
# 
'''
------------
Mode of use
------------
$ rosrun pybot pybot_camera
'''

__author__      = 'Juan David Adarve'
__email__       = 'juan.adarve@anu.edu.au'
__license__     = 'MIT'
__copyright__   = 'Copyright 2015, Australian Centre for Robotic Vision'

import threading
import rospy
import cv2
import numpy as np

from geometry_msgs.msg import Twist
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs.msg import Image, CompressedImage

class CameraPublisher(object):
    
    def __init__(self, topic='pybot/camera/image_color/compressed', camindex=0, resolution=(640,480)):
        
        # creates a ROS publisher for compressed images
        self.__publisher = rospy.Publisher(topic, CompressedImage)
        
        self.__capture = cv2.VideoCapture(camindex)
        if self.__capture == None:
            rospy.logerr('CameraPublisher: capture device not found')
            quit()
        else:
            rospy.loginfo('CameraPublisher: capture device found')
        
        self.__capture.set(cv2.cv.CV_CAP_PROP_FRAME_WIDTH, resolution[0])
        self.__capture.set(cv2.cv.CV_CAP_PROP_FRAME_HEIGHT, resolution[1])
    
        rospy.loginfo('CameraPublisher: starting capture loop')
        self.__imgThread = threading.Thread(target=self.__imageLoop)
        self.__imgThread.start()
    
    
    def __imageLoop(self):
        """
        Image acquisition and processing loop.
        
        This method constantly reads an image from the capture device and
        compresses it and publishes it in the ROS topic
        """
        
        # 10 Hz frame rate
        self.__rate = rospy.Rate(10)
        
        while not rospy.is_shutdown():
            
            try:
                # reads a new image from the camera
                self.__imgBGR = self.__capture.read()[1]
                
                if self.__imgBGR != None:
                    
                    # creates a compressed image message
                    imgMsg = CompressedImage()
                    imgMsg.header.stamp = rospy.Time.now()
                    imgMsg.format = 'jpeg'
                    imgMsg.data = np.array(cv2.imencode('.jpg', self.__imgBGR)[1]).tostring()
                     
                    # publish the image
                    self.__publisher.publish(imgMsg)
                    
                    cv2.imshow('camera', self.__imgBGR)
                    cv2.waitKey(10)
                    
                else:
                    rospy.logerr('CameraPublisher: error: no image read from camera')
                    
                self.__rate.sleep()
                
            except Exception as e:
                rospy.logerr('CameraPublisher: error reading image frame: {0}'.format(e.errmsg))

###########################################################
# ENTRY POINT
###########################################################
if __name__ == '__main__':
    
    # init ros node
    rospy.init_node('pybot_camera')
    
    rospy.loginfo('pybot_camera: start')
    
    # read node parameters
    topic = rospy.get_param('pybot_camera/topic', 'pybot/camera/image_color/compressed')
    camindex = rospy.get_param('pybot_camera/camera', 0)
    resolution = rospy.get_param('pybot_camera/resolution', [320, 240])
    
    rospy.loginfo('topic name: {0}'.format(topic))
    rospy.loginfo('camera index: {0}'.format(camindex))
    rospy.loginfo('image resolution: {0}'.format(resolution))
    
    webCamPub = CameraPublisher(topic, camindex, resolution)
    
    try:
        rospy.spin()
    except KeyboardInterrupt:
        rospy.loginfo('pybot_camera: keyboard interrupt, shutting down')
    
    cv2.destroyAllWindows()

### 3.2 Subscriber

In [ ]:
#!/usr/bin/python
#
# The MIT License (MIT)
# 
# Copyright (c) 2015 Australian Centre for Robotic Vision
# 

"""
pybot_vision: basic image receiver demo

This demo subscribes to an image topic and display its content in an
OpenCV window.

------------
Mode of use
------------
$ rosrun pybot pybot_vision _topic:=<image_topic> _compressed:=<bool>

_topic: image topic path, default 'camera/rgb/image_color/compressed'
_compressed: tell if topic is a compressed image topic, default true
"""

__author__      = 'Juan David Adarve'
__email__       = 'juan.adarve@anu.edu.au'
__license__     = 'MIT'
__copyright__   = 'Copyright 2015, Australian Centre for Robotic Vision'

import rospy
import cv2
import numpy as np

from geometry_msgs.msg import Twist
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs.msg import Image, CompressedImage


class ImageReceiver(object):
    """
    Image receiver class
    
    Objects of this class subscribe to a ROS image topic, either raw or compresssed,
    and call image listeners waiting for new image data to process.
    """
    
    def __init__(self, imageTopic, compressed=False):
        """
        Creates an image receiver subscribing to a specific image topic
        
        ----------------
        PARAMETERS
        ----------------
        imageTopic : string with the image topic to subscribe
        compressed : boolean to specify if the image topic is compressed (default False)
        """
        
        self.__compressedImage = compressed
        self.__imgListeners = list()
        
        if self.__compressedImage:
            self.__subscriber = rospy.Subscriber(imageTopic, CompressedImage, self.callback, queue_size=1)
        else:
            self.__bridge = CvBridge()
            self.__subscriber = rospy.Subscriber(imageTopic, Image, self.callback, queue_size=1)
        
        print('ImageReceiver: subscribed to topic {0}\tcompresssed: {1}'.format(imageTopic, compressed))
    
        
    def callback(self, imgObj):
        """
        Image callback
        
        Each time a new image arrives, all image listeners are called with the
        new image as parameter.
        """
        
        if self.__compressedImage:
            arrData = np.fromstring(imgObj.data, np.uint8)
            self.image = cv2.imdecode(arrData, cv2.CV_LOAD_IMAGE_COLOR)
        else:
            try:
                self.image = self.__bridge.imgmsg_to_cv2(imgObj, 'passthrough')
            except CvBridgeError as e:
                print('ImageReceiver: error receiving image (uncompressed): {0}'.format(e))
        
        # broadcast the new image to all listeners
        for l in self.__imgListeners:
            try:
                l(self.image)
            except Exception as e:
                print('ImageReceiver: error broadcasting image: {0}'.format(e))


    def addImageListener(self, listener):
        """
        Add a new imageListener
        
        ----------------
        PARAMETERS
        ----------------
        listener : callable object with signature method(img)
        """
        self.__imgListeners.append(listener)



class ImageViewer(object):
    """
    Image viewer class
    
    Objects of this class receive an image through the imageCallback function
    and display it on screen using OpenCV imshow method.
    """
    
    def __init__(self):
        self.title = 'image'
    
    def imageCallback(self, img):
        """
        Image callback called by the imageReceiver
        
        ----------------
        PARAMETERS
        ----------------
        img : new image data
        """
        cv2.imshow(self.title, img)
        cv2.waitKey(10)

            
###########################################################
# ENTRY POINT
###########################################################
if __name__ == '__main__':
    
    print('pybot_vision: start')
    
    # init ros node
    rospy.init_node('pybot_vision')
    
    topic = rospy.get_param('pybot_vision/topic', 'camera/rgb/image_color/compressed')
    compressed = rospy.get_param('pybot_vision/compressed', True)
    
    # creates an image receiver remotely connected to the image_color topic
    imgReceiver = ImageReceiver(topic, compressed=compressed)

    # creates an image viewer and subscribe it to the callback list of imgRecevier
    imgViewer = ImageViewer()
    imgReceiver.addImageListener(imgViewer.imageCallback)
    
    try:
        rospy.spin()
    except KeyboardInterrupt:
        print('pybot_vision: keyboard interrupt, shutting down')
    
    cv2.destroyAllWindows()

## Compile and run
**`
cd ~/workspace_ros
catkin_make
source devel/setup.bash
rosrun pybot pybot_camera.py
`**

Open new terminal window.

**`
rosrun pybot pybot_vision.py _topic:=/pybot/camera/image_color/compressed
`**
